In [1]:
#paths required
datafolder_p = r'D:\18-DS\github\SDSHL\data\processed'
datafolder_e = r'D:\18-DS\github\SDSHL\data\external'
datafolder_i = r'D:\18-DS\github\SDSHL\data\internal'
modelfolder  = r'D:\18-DS\data\models\POS-Tagger-Hindi\hi_hdtb_models'

# filepath_train =datafolder_p+ r'\2-train.csv'
# filepath_test  =datafolder_p+ r'\2-test.csv'

filepath_data = datafolder_p + r'\2-Hinglish_Sarcasm_Clean.csv'

In [ ]:
#!pip install stanfordnlp
import stanfordnlp
#stanfordnlp.download('hi')

In [2]:
import pandas as pd
df = pd.read_csv(filepath_data, sep="\t", index_col="ID")

In [3]:
docs = list(df['sentence_wo_emo'])
#use sentence_wo_emo to perform pos

In [4]:
tag = modelfolder+ r'\hi_hdtb_tagger.pt'
pret = modelfolder+ r'\hi_hdtb.pretrain.pt'

config = {
        'processors': 'tokenize,pos',
        'tokenize_pretokenized': True,
        'pos_model_path': tag,
        'pos_pretrain_path': pret,
        'pos_batch_size': 1000,
        'lang':'hi'
         }
nlp = stanfordnlp.Pipeline(**config)

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': 'C:\\Users\\admin\\stanfordnlp_resources\\hi_hdtb_models\\hi_hdtb_tokenizer.pt', 'pretokenized': True, 'lang': 'hi', 'shorthand': 'hi_hdtb', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': 'D:\\18-DS\\data\\models\\POS-Tagger-Hindi\\hi_hdtb_models\\hi_hdtb_tagger.pt', 'pretrain_path': 'D:\\18-DS\\data\\models\\POS-Tagger-Hindi\\hi_hdtb_models\\hi_hdtb.pretrain.pt', 'batch_size': 1000, 'lang': 'hi', 'shorthand': 'hi_hdtb', 'mode': 'predict'}
Done loading processors!
---


### Create POS for each sentence

In [5]:
#This step is TIME Consuming

doc_pos=[]
for i in range( len( docs ) ):
    doc_nlp = nlp( docs[i] ) #get nlp of ith sentence
    no_of_words = len(doc_nlp.conll_file.sents[0])
    pos_list =[]
    for j in range( no_of_words ):
        pos = doc_nlp.conll_file.sents[0][j][3] # 3rd is POS name
        pos_list.append(pos)
    pos_list = ", ".join(pos_list)
    doc_pos.append(pos_list)

### create a dataset of pos

In [6]:
df_pos = pd.DataFrame(doc_pos, columns=['pos'], index=df.index)
df_pos['sentence_wo_emo']=list(docs)

### unique pos in our dataset

In [7]:
pos_types=[]
pos_sents= df_pos.pos.str.replace(" ","").str.split(",")
for pos_sent in pos_sents:  
    for pos_word in pos_sent:
        if pos_word not in pos_types:
            pos_types.append (pos_word)

print((pos_types))

['NOUN', 'ADP', 'VERB', 'AUX', 'PRON', 'PROPN', 'PART', 'DET', 'ADJ', 'SCONJ', 'PUNCT', 'CCONJ', 'NUM', 'ADV', 'INTJ', 'X']


### get frequencies of pos from words in sentences

In [8]:
import numpy as np
rows = len(df_pos)
cols = len(pos_types) + 1 # one more colmn for number of words in sentence
pos_vec= np.zeros((rows, cols), dtype=int)

i=0
for i in range(len(df_pos)):
    sent    = df_pos.iloc[i,]['sentence_wo_emo']
    pos_list= df_pos.iloc[i,]['pos'].replace(" ","").split(",")
    for j in range(len(pos_types)):
        pos1=pos_types[j]
        find_pos= [pos1]*len(pos_list)       
        pos_vec[i][j]= sum(np.array(pos_list)==np.array(find_pos))
    
    pos_vec[i][j+1]=len(pos_list)

In [9]:
#save pos frequencies in a dataset
col_names = pos_types.copy()
col_names.append('words')
df_pos1 = pd.DataFrame(pos_vec, columns=col_names, index=df_pos.index)

#merge the pos dataset with original dataset
df_pos1 = pd.concat( [df_pos1, df[['label','sentence_org','sentence','sentence_wo_emo']], df_pos['pos'] ], axis=1, join="inner"  )

In [10]:
df_pos1.shape

(2000, 22)

In [11]:
#save pos file
df_pos1.to_csv(datafolder_p+ r'\4-features_pos.csv',sep='\t', encoding="utf-8")

# <font color=red> Experiment Area </font>

In [ ]:
docs = df['sentence']
#use regex and remove emoticons

In [ ]:
import stanfordnlp
folder=r'D:\18-DS\data\models\POS-Tagger-Hindi'

tag = folder+ r'\hi_hdtb_models\hi_hdtb_tagger.pt'
pret = folder+ r'\hi_hdtb_models\hi_hdtb.pretrain.pt'


#'pos_model_path': './en_ewt_models/en_ewt_tagger.pt',
#'pos_pretrain_path': './en_ewt_models/en_ewt.pretrain.pt',

config = {
        'processors': 'tokenize,pos',
        'tokenize_pretokenized': True,
        'pos_model_path': tag,
        'pos_pretrain_path': pret,
        'pos_batch_size': 1000,
        'lang':'hi'
         }
nlp = stanfordnlp.Pipeline(**config)
# doc = nlp("""केंद्र की मोदी सरकार ने शुक्रवार को अपना अंतरिम बजट पेश किया.""")

# # doc = nlp("""केंद्र की मोदी सरकार ने शुक्रवार को अपना अंतरिम बजट पेश किया. कार्यवाहक वित्त मंत्री पीयूष गोयल \
# # ने अपने बजट में किसान, मजदूर, करदाता, महिला वर्ग समेत हर किसी के लिए बंपर ऐलान किए. हालांकि, \
# # बजट के बाद भी टैक्स को लेकर काफी कन्फ्यूजन बना रहा. \
# # केंद्र सरकार के इस अंतरिम बजट क्या खास रहा और किसको क्या मिला, आसान भाषा में यहां समझें""")
# print(doc.conll_file.conll_as_string())

In [ ]:
doc_pos=[]
for i in range( len( docs ) ):
    doc = nlp( docs[i] )
    pos_list =[]
    for j in range( len(doc.conll_file.sents[0]) ):
        pos = doc.conll_file.sents[0][j][3]
        #print (doc.conll_file.sents[0][j][1] , pos)
        pos_list.append(pos)
    pos_list = ", ".join(pos_list)
    doc_pos.append(pos_list)
    

In [ ]:
len(doc_pos)

In [ ]:
train_pos = pd.DataFrame(doc_pos, columns=['pos'], index=df['ID'])

In [ ]:
train_pos

In [ ]:
#len(train_pos.iloc[1,].pos.split(","))

pos1

In [ ]:
pos1=(train_pos.iloc[1,].pos.split(","))
x = filter('NOUN', pos1) 

In [ ]:
print (pos1)
for i in x:
    print (i)

In [ ]:

def find_pos( pos_list): 
    if (pos2_name in pos_list ): 
        return True
    else: 
        return False
  
  
# using filter function 
pos2_name='NOUN'
filtered = filter(find_pos,  pos1) 
#print (list(filtered))  
#print('The filtered letters are:') 

for s in filtered: 
    print(s) 

In [ ]:
import numpy as np
a=np.array(pos1)==np.array(['NOUN']*len(pos1))

pos1=(train_pos.iloc[1,].pos.split(","))
print (pos1)
print (np.array(['NOUN']*len(pos1)))
print (np.array(pos1))
print (a)

In [ ]:
def myFun(**kwargs): 
    for key, value in kwargs.items():
        print ("%s == %s" %(key, value))
 
# Driver code
myFun(first ='Geeks', mid ='for', last='Geeks')   

In [ ]:
np.array( ['NOUN']*len(pos1) ) == np.array( pos1 )

In [ ]:
pos1

In [ ]:
np.array( pos1, dtype="<U6" )

In [ ]:
np.array( ['NOUN']*len(pos1), dtype='<U6' )

In [ ]:
np.array( pos1, dtype="<U6" ) ==np.array( ['NOUN']*len(pos1), dtype='<U6' )

In [ ]:
pos_list=(train_pos.iloc[1,].pos.split(","))
for i in range(len(pos_list)):
    print ( pos_list[i])

In [ ]:
x=str(train_pos.iloc[1,]).split(",")
x

In [ ]:
train_pos